Import the necessary packages

In [1]:
#!pip install package

import numpy as np
import pandas as pd
import tweepy
import pprint, time
from math import ceil

Import the original csv file from the GitHub account, which is in this folder.

In [2]:
hs = pd.read_csv('NAACL_SRW_2016.csv', index_col=0, header=None)
hs = hs.reset_index()

Rename column headings, reset ID to not be the ID number

In [3]:
hs = hs.rename(columns={0: 'ID', 1: 'Class'})
hs.Class = hs.Class.astype('str')
print(hs.head())

                   ID   Class
0  572342978255048705  racism
1  572341498827522049  racism
2  572340476503724032  racism
3  572334712804384768  racism
4  572332655397629952  racism


Determine how big the dataset is (rows, columns)

In [38]:
len(hs)

16907

Query the Twitter API to get the tweets from the IDS. Get various key's from Twitter developer website (dev.twitter.com). Run 167 loops of 100 tweets each: first, subset the dataframe by the 100 tweets of each iteration (line 14). Then, define an object "tweets" which actually queries the Twitter API for the IDs (fed in list form) of that subset of tweets. Then add those tweets to an existing list, tweet_list, initialized to an empty list in line 10. Finally, sleep for 60 seconds between each loop to avoid breaking the API rate limit. 

In [1]:
####CHANGE THE KEYS TO YOUR OWN USING THE TWITTER API####

consumer_key = "Use Yours"
consumer_secret = "Use Yours"
access_token = "Use Yours"
access_token_secret = "Use Yours"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

tweet_list = []


for i in range(0, ceil(len(hs.ID) / 100)):
    ids = hs.ID.iloc[i*100 : (i+1)*100]
    tweets = api.statuses_lookup(list(ids), True)
    tweet_list.extend(list(tweets))
    time.sleep(60)

NameError: name 'tweepy' is not defined

Choose one tweet and print an object from that tweet for testing purposes, here the first tweet t and its object "text." 

In [12]:
t = tweet_list[0]
len(tweet_list)

16035

After you've run the querying code for at least three hours, you can create lists of various objects from the tweets. Here is how you do that for the tweets:

In [17]:
t= tweet_list[0]

tweets = []

for i in range(16035):
	t = tweet_list[i]
	tweets.append(t.text)
    
#print(tweets)

                                              Tweets
0  @ummayman90 @wood_brwood333 For example, in Me...
1  @ummayman90 Again, your entire concept of god ...
2  @anjemchoudary Your prophet was a rapist, murd...
3  RT @DilanaKurdi: Yazidi children who are taken...
4  @anjemchoudary Accepting the idea that Allah o...


And authors. Here I've also included code for how to write the list to a csv file so that we can use it later.

In [19]:
authors = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	authors.append(t.author.screen_name)

#print(authors)

Locations:

In [21]:
locations = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	locations.append(t.author.location)

#print(locations)

"Descriptions" - Twitter bios:

In [23]:
descriptions = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	descriptions.append(t.author.description)

#print(descriptions)

Number of followers:

In [25]:
followers_count = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	followers_count.append(t.author.followers_count)

#print(followers_count)

Time tweeted:

In [28]:
time_tweeted = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	time_tweeted.append(t.created_at)

#print(time_tweeted)

User mentions:

In [30]:
user_mentions = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	user_mentions.append(t.entities['user_mentions'])
#print(user_mentions)

Hashtags:

In [33]:
hashtags = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	hashtags.append(t.entities['hashtags'])

#print(hashtags)

Favorite counts:

In [35]:
favorite_counts = []
t=tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	favorite_counts.append(t.favorite_count)

#print(favorite_counts)

In [45]:
id_str = []
t = tweet_list[0]

for i in range(16035):
	t = tweet_list[i]
	id_str.append(t.id_str)
    
#print(id_str)

In [46]:
tweet_info = pd.DataFrame({'Tweets': tweets, "ID": id_str, "Authors": authors, "Locations": locations, "Descriptions": descriptions,
                          "Follower Count":followers_count, "Time Tweeted": time_tweeted, "User Mentions": user_mentions, 
                           "Hashtags": hashtags, "Favorite Count":favorite_counts})
#print(tweet_info.head())

In [43]:
tweet_info.to_csv('hatespeech.csv')